In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler,MaxAbsScaler
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow.keras import optimizers
from tensorflow.keras import losses

# Load CSV data
data = pd.read_csv('heart_attack_prediction_final.csv')

#convert target to binary label
data['target'] = (data['target'] >= 0.5).astype(int)
data.head()

# Assuming the last column is the target variable and rest are features
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, shuffle=True)
X_train.shape
X_test.shape
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Logistic Regression Model
LR_model = LogisticRegression(random_state=42)
LR_model.fit(X_train, y_train)

LR_pred = LR_model.predict(X_test)

LR_accuracy = accuracy_score(y_test, LR_pred)
LR_f1_score = f1_score(y_test, LR_pred)
LR_precision = precision_score(y_test, LR_pred)
LR_recall = recall_score(y_test, LR_pred)

#NN Classifier
inputs = layers.Input(shape = (13,), name = 'Input_layer')

dense_1 = layers.Dense(10, kernel_regularizer = regularizers.l2(0.01),activation = 'relu', name = 'Dense1')(inputs)
batch_normal1 = layers.BatchNormalization(name = 'Batch_normal1')(dense_1)
dropout_1 = layers.Dropout(0.3 , name = 'Dropout1')(batch_normal1)

dense_2 = layers.Dense(20, kernel_regularizer = regularizers.l2(0.01), activation = 'relu', name = 'Dense2')(dropout_1)
batch_normal2 = layers.BatchNormalization(name = 'Batch_normal2')(dense_2)
dropout_2 = layers.Dropout(0.3, name = 'Dropout2')(batch_normal2)

dense_3 = layers.Dense(40, kernel_regularizer = regularizers.l2(0.01), activation = 'relu', name = 'Dense3')(dropout_2)
batch_normal3 = layers.BatchNormalization(name = 'Batch_normal3')(dense_3)
dropout_3 = layers.Dropout(0.3, name = 'Dropout3')(batch_normal3)

dense_4 = layers.Dense(80, kernel_regularizer = regularizers.l2(0.01), activation = 'relu', name = 'Dense4')(dropout_3)

outputs = layers.Dense(1, activation = 'sigmoid', name = 'output')(dense_4)

#NN_model.summary()

NN_model = tf.keras.Model(inputs = inputs, outputs = outputs)
NN_model.compile(loss = losses.BinaryCrossentropy(), optimizer = optimizers.Adam(learning_rate = 0.05), metrics = ['accuracy'])

#callbacks to reduce overfitting, add , callbacks = [early_stopping, lr_scheduler] to .fit command when in use
early_stopping= keras.callbacks.EarlyStopping(monitor='val_accuracy', mode='auto', patience=20, restore_best_weights=True)
lr_scheduler = keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy',mode='auto',factor=0.5,patience=5)

history = NN_model.fit(X_train, y_train, epochs = 150, shuffle = True, validation_data = (X_test,y_test), batch_size = 32, callbacks = [early_stopping, lr_scheduler])
NN_model.evaluate(X_test, y_test)

NN_pred = NN_model.predict(X_test)
NN_pred = np.round(NN_pred)

NN_accuracy = accuracy_score(y_test, NN_pred)
NN_f1_score = f1_score(y_test, NN_pred)
NN_precision = precision_score(y_test, NN_pred)
NN_recall = recall_score(y_test, NN_pred)

#KNN classifier
knn_classifier = KNeighborsClassifier(n_neighbors=10, metric='cosine')
knn_classifier.fit(X_train, y_train)
knn_predictions = knn_classifier.predict(X_test)

knn_accuracy = accuracy_score(y_test, knn_predictions)
knn_precision = precision_score(y_test, knn_predictions)
knn_recall = recall_score(y_test, knn_predictions)
knn_f1_score = f1_score(y_test, knn_predictions)

# Decision Tree Classifier
dt_classifier = DecisionTreeClassifier()
dt_classifier.fit(X_train, y_train)

dt_predictions = dt_classifier.predict(X_test)
dt_accuracy = accuracy_score(y_test, dt_predictions)
dt_precision = precision_score(y_test, dt_predictions)
dt_recall = recall_score(y_test, dt_predictions)
dt_f1_score = f1_score(y_test, dt_predictions)

# Random Forest Classifier
rf_classifier = RandomForestClassifier()
rf_classifier.fit(X_train, y_train)

rf_predictions = rf_classifier.predict(X_test)
rf_accuracy = accuracy_score(y_test, rf_predictions)
rf_precision = precision_score(y_test, rf_predictions)
rf_recall = recall_score(y_test, rf_predictions)
rf_f1_score = f1_score(y_test, rf_predictions)

print()
print("LOGISTIC REGRESSION MODEL")
print("-----------------------------------")
print("LR Accuracy:", LR_accuracy)
print("LR Precision:", LR_precision)
print("LR Recall:", LR_recall)
print("LR F1 Score:", LR_f1_score)
print()

print("NN MODEL")
print("-----------------------------------")
print("NN Accuracy:", NN_accuracy)
print("NN Precision:", NN_precision)
print("NN Recall:", NN_recall)
print("NN F1 Score:", NN_f1_score)
print()

print("KNN MODEL")
print("-----------------------------------")
print("KNN Accuracy:", knn_accuracy)
print("KNN Precision:", knn_precision)
print("KNN Recall:", knn_recall)
print("KNN F1 Score:", knn_f1_score)
print()

print("DECISION TREE CLASSIFIER")
print("-----------------------------------")
print("Decision Tree Accuracy:", dt_accuracy)
print("Decision Tree Precision:", dt_precision)
print("Decision Tree Recall:", dt_recall)
print("Decision Tree F1 Score:", dt_f1_score)
print()

print("RANDOM FOREST CLASSIFIER")
print("-----------------------------------")
print("Random Forest Accuracy:", rf_accuracy)
print("Random Forest Precision:", rf_precision)
print("Random Forest Recall:", rf_recall)
print("Random Forest F1 Score:", rf_f1_score)